In [ ]:
import os
import json

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import spacy

from transformers import AutoModelForTokenClassification, AutoTokenizer

import matplotlib.pyplot as plt

In [ ]:
from training_utils import load_data, read_entity_dict, spacy_to_biobert_input, compute_ner_metrics

In [ ]:
from extraction_utils import *

## Constants
*Defining constants upfront for easy configuration and maintenance.*

In [ ]:
DATA_PATH = '/data/02_training_data/'
SCISPACY_MODEL_NAME = "/data/03_models/finetuned/scispacy/finetuned_scispacy_ner/"
BIOBERT_MODEL_NAME = "/data/03_models/finetuned/biobert/finetuned_biobert_ner/"

## Dataset Preparation
### Load Data

In [ ]:
try:
    test = load_data(os.path.join(DATA_PATH,'temp_test_NER.pickle'))
except FileNotFoundError as e:
    print(e)

### Load Label Mappings

In [ ]:
# Loading label_to_id from a JSON file
with open(os.path.join(DATA_PATH, 'label_to_id.json'), 'r') as file:
    label_to_id = json.load(file)

# Loading id_to_label from a JSON file
with open(os.path.join(DATA_PATH, 'id_to_label.json'), 'r') as file:
    id_to_label = json.load(file)
id_to_label = {int(k): v for k, v in id_to_label.items()}

In [ ]:
label_to_id

In [ ]:
id_to_label

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained(BIOBERT_MODEL_NAME)

### Translate Spacy Docs to BioBert input format

In [ ]:
test_data_biobert = spacy_to_biobert_input(test, bert_tokenizer, label_to_id)

# Load Models

In [ ]:
bert_model = AutoModelForTokenClassification.from_pretrained(BIOBERT_MODEL_NAME)

In [ ]:
nlp = spacy.load(SCISPACY_MODEL_NAME)  # Choose the appropriate SciSpacy model

### Make predictions with both models

In [ ]:
biobert_preds = [biobert_entity_predict_conditional_window(t.text, model=bert_model, tokenizer=bert_tokenizer, id_to_label=id_to_label) for t in test]
scispacy_preds = [scispacy_predict(t.text, nlp=nlp) for t in test]

In [ ]:
combined_preds = merge_predictions_with_sciSpacy_priority(biobert_preds, scispacy_preds)

### Combine Predictions

## Evaluation
### Normalize entity tags

In [ ]:
test_labels = [[(e.start_char, e.end_char, e, e.label_) for e in t.ents] for t in test]

In [ ]:
combined_preds = [predictions_to_tags(test[i], doc_preds) for i, doc_preds in enumerate(combined_preds)]
test_labels = [predictions_to_tags(test[i], doc_labels) for i, doc_labels in enumerate(test_labels)]

In [ ]:
metrics = compute_ner_metrics([pred for doc in combined_preds for pred in doc], [pred for doc in test_labels for pred in doc])

print("Precision:", metrics["precision"])
print("Recall:", metrics["recall"])
print("F1-Score:", metrics["f1"])
print("\nClassification Report:\n", metrics["classification_report"])

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix([pred for doc in combined_preds for pred in doc],  [pred for doc in test_labels for pred in doc], labels=['O', 'TARGETGENE', 'SRNA'])
# Plotting the confusion matrix
fig, ax = plt.subplots(figsize=(10, 8))
plt.xlabel('Predicted Labels')
plt.ylabel('Ground Truth Labels')
plt.title('Ensemble NER Label-level Confusion Matrix')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['O', 'TARGETGENE', 'SRNA'])
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.show()